In [17]:
import pandas as pd
import re
import json

In [18]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [19]:
with open("../data/bronze/Details.json", 'r') as json_file:
    data = json.load(json_file)

In [29]:
data

{'Aloe': {'Overview': 'Aloe (aloe vera), is a cactus-like plant that grows in hot, dry climates. The useful parts of aloe are the gel and latex.\n\nAloe gel might help some skin conditions like psoriasis. It also seems to speed up wound healing by improving blood circulation, and might combat certain types of bacteria and fungi. Aloe latex contains chemicals that work as a laxative. Some aloe products are made from the whole crushed leaf, so they contain both gel and latex.\n\nPeople commonly apply aloe gel to the skin for conditions such as sunburn, acne, dandruff, and many others, but there is no good scientific evidence to support these uses. Some people take aloe products by mouth for conditions such as obesity, diabetes, osteoarthritis, and many others, but there is also no good scientific evidence to support these uses.',
  'Uses': {'Possibly Effective for': [['Acne. Applying an aloe gel to the skin in the morning and evening, together with a prescription anti-acne medicine, can 

## FUNCTIONS

In [28]:
def get_number(text: str):
    cleaned = re.findall(r'\d+', text)[0]
    return int(cleaned)

In [ ]:
def get_reviews(url):
    
    return reviews

In [30]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
waitTime = WebDriverWait(driver,10)

for supplement, information in data.items():
    numReviews = get_number(information["Reviews"]['NumberOfReviews'])
    url = information["Reviews"]['Url']

    if numReviews == 0:
        continue

    driver.get(url)
    